In [1]:
from pyspark.sql import SparkSession
import requests
import pandas as pd
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import DataType, StructType, StructField, StringType, IntegerType, DoubleType

In [2]:
spark = SparkSession.builder.getOrCreate()
url = "https://maps.stlouisco.com/arcgis/rest/services/OpenData/OpenData/FeatureServer/7/query?where=1%3D1&outFields=*&geometry=-90.454%2C38.723%2C-90.381%2C38.746&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outSR=4326&f=json"
s = requests.get(url).json()
df = pd.json_normalize(s, record_path="features")
df.columns = df.columns.str.lstrip('attributes.')

In [3]:
#df.printSchema()
schema = StructType()
for feature in s['fields']:
    if feature['type'] == 'esriFieldTypeString':
        schema.add(feature['name'], StringType(), True)
    elif feature['type'] == 'esriFieldTypeSmallInteger':
        schema.add(feature['name'], IntegerType(), True)
    elif feature['type'] == 'esriFieldTypeDouble':
        schema.add(feature['name'], StringType(), True)
    else:
        schema.add(feature['name'], StringType(), True)
schema = schema.add("geometry.rings", StringType(), True)
    

In [4]:
sdf = spark.createDataFrame(data=df, schema=schema)

In [5]:
sdf.printSchema()

root
 |-- OBJECTID: string (nullable = true)
 |-- PARENT_LOC: string (nullable = true)
 |-- LOCATOR: string (nullable = true)
 |-- TAXYR: integer (nullable = true)
 |-- OWNER_NAME: string (nullable = true)
 |-- PROP_ADRNUM: string (nullable = true)
 |-- PROP_ADD: string (nullable = true)
 |-- PROP_ZIP: string (nullable = true)
 |-- OWN_ADD: string (nullable = true)
 |-- OWN_CITY: string (nullable = true)
 |-- OWN_STATE: string (nullable = true)
 |-- OWN_ZIP: string (nullable = true)
 |-- SCHSUB: string (nullable = true)
 |-- MUNYCODE: string (nullable = true)
 |-- SUBDIVISION: string (nullable = true)
 |-- TAXCODE: string (nullable = true)
 |-- ASSTLANDVAL: string (nullable = true)
 |-- ASSTIMPVAL: string (nullable = true)
 |-- TOTASSMT: string (nullable = true)
 |-- APPLANDVAL: string (nullable = true)
 |-- APPIMPVAL: string (nullable = true)
 |-- TOTAPVAL: string (nullable = true)
 |-- PROPCLASS: string (nullable = true)
 |-- LUC: string (nullable = true)
 |-- LANDUSE2: string (nulla